<a href="https://colab.research.google.com/github/akiran703/DQN_demo_for_SI/blob/main/DQN_for_spaceinvaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]
!pip install stable-baselines3[extra]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.8 MB/s eta 0:00:00
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2

In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from gym import spaces

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2

cv2.ocl.setUseOpenCL(False)


import stable_baselines3


In [2]:
from ale_py import ALEInterface
ale = ALEInterface()

from ale_py.roms import SpaceInvaders
ale.loadROM(SpaceInvaders)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
#in order to reduce instability for when the Deep Q-learning is being trained, we apply a replay system to help.
#A replay system essentially is a way to store transistions that the agent observes and it can be used later training.
#This helps with reducing catostraphic forgetting in which the agent forgets all previous experiences as its new experience and reduces correlation between experiences.



#creating that class that creates the Replay system
class ReplayMemory:
    #constructor creating a list to store the experiences
    def __init__(self, capacity):
        self.memory = []
        self.maxlen=capacity
        self.nextindex = 0

    # we can not keep every single transition, so we will remove the oldest one
    def push(self, state,action,reward,next_state,done):

      data = (state, action, reward, next_state, done)

      if self.nextindex >= len(self.memory):
        self.memory.append(data)
      else:
        self.memory[self.nextindex] = data
      self.nextindex = (self.nextindex + 1) % self.maxlen


    #encode the sample into arrays
    def encode_sample(self,indices):
      states, actions, rewards, next_states, dones = [], [], [], [], []
      for i in indices:
        d = self.memory[i]
        state, action, reward, next_state, done = d
        states.append(np.array(state,copy=False))
        actions.append(action)
        rewards.append(reward)
        next_states.append(np.array(next_state, copy=False))
        dones.append(done)
      return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)




    # getting a random experience
    def sample(self, batch_size):
      indx = np.random.randint(0, len(self.memory) - 1, size = batch_size)
      return self.encode_sample(indx)


    # the length of the memory
    def __len__(self):
        return len(self.memory)


In [4]:
class DQN(nn.Module):
  def __init__(self,observation_space: spaces.Box, action_space: spaces.discrete):
     super().__init__()
     # checks to enable that the observation space is a box, it forms a channels x widith x height, the action space is discrete
     assert type(
            observation_space) == spaces.Box, 'observation_space must be of type Box'
     assert len(
            observation_space.shape) == 3, 'observation space must have the form channels x width x height'
     assert type(
            action_space) == spaces.Discrete, 'action_space must be of type Discrete'

      #Convolution layer with activation function
     self.conv = nn.Sequential(
         nn.Conv2d(in_channels=observation_space.shape[0], out_channels=32, kernel_size=8, stride=4),
         nn.ReLU(),
         nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
         nn.ReLU(),
         nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
         nn.ReLU()
     )

     #fully connected layer
     self.fc = nn.Sequential(
            nn.Linear(in_features=64*7*7 , out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=action_space.n)
        )
  #feed foward neural network
  def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0],-1)
        return self.fc(conv_out)

In [17]:
#creating the agent class which contains double DQN and

class DQN_Agent:
  def __init__(self,observation_space: spaces.Box, action_space: spaces.discrete,replay_buffer: ReplayMemory,use_double_dqn,lr,batch_size,gamma,device=torch.device('cpu')):
    self.memory = replay_buffer
    #double dqn to make sure the estimated Q-value isnt overestimated
    self.use_double_dqn = use_double_dqn
    self.batch_size = batch_size
    self.gamma = gamma



    self.policy_network = DQN(observation_space,action_space).to(device)
    self.target_network = DQN(observation_space,action_space).to(device)
    self.update_target_network()
    self.target_network.eval()


    #in order minimize our loss function between the estimated Q-value and Q-target value, we use gradient decsent with Root Mean Square propagation
    #gradient decsent with Root Mean Square propagation esstentially takes the learning rate and expotentially decays it when the squared gradient is less than a threshold
    #we do this to ensure each parameter in the model is adjusted thus providing better peformance
    #step 1: set the parameters Learning rate: α, Exponential decay rate for averaging: γ, Small constant for numerical stability: ε, Initial parameter values: θ
    #step 2: initalize the accumulated gradients
    #step 3 update the parameters after calculating the gradient of the objective function with respect to the parameters and updating exponentially weighted average of the squared gradients
    self.optimiser = torch.optim.RMSprop(self.policy_network.parameters()
            , lr=lr)


    self.device = device

  def update_target_network(self):

    self.target_network.load_state_dict(self.policy_network.state_dict())

  def temporaldifference_loss(self):
    device = self.device


    states, actions, rewards, next_states, dones = self.memory.sample(self.batch_size)
    states = np.array(states) / 255.0
    next_states = np.array(next_states) / 255.0
    states = torch.from_numpy(states).float().to(device)
    actions = torch.from_numpy(actions).long().to(device)
    rewards = torch.from_numpy(rewards).float().to(device)
    next_states = torch.from_numpy(next_states).float().to(device)
    dones = torch.from_numpy(dones).float().to(device)


    #double DQN uses to networks, a target network and a policy network
    # policy network will choose the highest Q-value and that will be our action
    # target network to calculate the  Q-value of taking that action in the next state
    with torch.no_grad():
      _, max_next_action = self.policy_network(next_states).max(1)
      estimated_q_value = self.target_network(next_states).gather(1,max_next_action.unsqueeze(1)).squeeze()
      target_q_value = rewards + (1-dones) * self.gamma * (estimated_q_value)

    pre_q_value = self.policy_network(states)
    pre_q_value = pre_q_value.gather(1,actions.unsqueeze(1)).squeeze()

    td_loss = F.smooth_l1_loss(pre_q_value, target_q_value)


    self.optimiser.zero_grad()
    td_loss.backward()
    self.optimiser.step()
    del states
    del next_states
    return td_loss.item()

  def act(self,state: np.ndarray):
    #using greedy policy to choose the action

    device = self.device
    state = np.array(state) / 255.0
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    with torch.no_grad():
      q_values = self.policy_network(state)
      _, action = q_values.max(1)
      return action.item()












In [8]:
class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.integers(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, info = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs, info

    def step(self, ac):
        return self.env.step(ac)


class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,) + env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work.
        Expects inputs to be of shape height x width x num_channels
        """
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
                                            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]


class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        Expects inputs to be of shape num_channels x height x width.
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0] * k, shp[1], shp[2]), dtype=np.uint8)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=0)
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._frames)

    def __getitem__(self, i):
        return self._frames[i]


class PyTorchFrame(gym.ObservationWrapper):
    """Image shape to num_channels x height x width"""

    def __init__(self, env):
        super(PyTorchFrame, self).__init__(env)
        shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(shape[-1], shape[0], shape[1]), dtype=np.uint8)

    def observation(self, observation):
        return np.rollaxis(observation, 2)

In [ ]:

if __name__ == '__main__':
    hyper_params = {
        "seed": 1,  # which seed to use
        "env": "ALE/SpaceInvaders-v5",  # name of the game
        "replay-buffer-size": int(5e3),  # replay buffer size
        "learning-rate": 1e-4,  # learning rate for Adam optimizer
        "discount-factor": 0.99,  # discount factor
        "num-steps": int(1e6),   # total number of steps to run the environment for
        "batch-size": 32,  # number of transitions to optimize at the same time
        "learning-starts": 10000,  # number of steps before learning starts
        "learning-freq": 1,  # number of iterations between every optimization step
        "use-double-dqn": True,  # use double deep Q-learning
        "target-update-freq": 1000,  # number of iterations between every target network update
        "eps-start": 1,  # e-greedy start threshold
        "eps-end": 0.01,  # e-greedy end threshold
        "eps-fraction": 0.1,  # fraction of num-steps
        "print-freq": 10
    }




    np.random.seed(hyper_params['seed'])
    random.seed(hyper_params['seed'])


    #make env
    env = gym.make(hyper_params['env'])
    env.seed(hyper_params['seed'])


    # sample inital states by taking random number of no-ops on reset
    env = NoopResetEnv(env, noop_max=30)
    # return every nth-skip frame
    env = MaxAndSkipEnv(env, skip=4)
    # make end-of-life == end-of-episode
    env = EpisodicLifeEnv(env)
    # take action on reset for env that are fixed until firing
    env = FireResetEnv(env)
    # make dimensions into 84 x 84 and make rbg to gray
    env = WarpFrame(env)
    # image shape to num_channels x height x widith
    env = PyTorchFrame(env)
    #[1,0,-1]
    env = ClipRewardEnv(env)
    #get the last k frames and stack
    env = FrameStack(env, 4)


    replay_buffer = ReplayMemory(hyper_params['replay-buffer-size'])

    agent = DQN_Agent(
        env.observation_space,
        env.action_space,
        replay_buffer,
        use_double_dqn=hyper_params["use-double-dqn"],
        lr=hyper_params['learning-rate'],
        batch_size=hyper_params['batch-size'],
        gamma=hyper_params['discount-factor'],
        device=torch.device("cpu")
    )


    eps_timesteps = hyper_params["eps-fraction"] * \
        float(hyper_params["num-steps"])
    episode_rewards = [0.0]


    state = env.reset()
    for t in range(hyper_params["num-steps"]):
        fraction = min(1.0, float(t) / eps_timesteps)
        eps_threshold = hyper_params["eps-start"] + fraction * \
            (hyper_params["eps-end"] - hyper_params["eps-start"])
        sample = random.random()

        if(sample > eps_threshold):
            # Exploit
            action = agent.act(state)
        else:
            # Explore
            action = env.action_space.sample()

        next_state, reward, done, info = env.step(action)
        agent.memory.push(state, action, reward, next_state, float(done))
        state = next_state

        episode_rewards[-1] += reward
        if done:
            state = env.reset()
            episode_rewards.append(0.0)

        if t > hyper_params["learning-starts"] and t % hyper_params["learning-freq"] == 0:
            agent.temporaldifference_loss()

        if t > hyper_params["learning-starts"] and t % hyper_params["target-update-freq"] == 0:
            agent.update_target_network()

        num_episodes = len(episode_rewards)

        if done and hyper_params["print-freq"] is not None and len(episode_rewards) % hyper_params[
                "print-freq"] == 0:
            mean_100ep_reward = round(np.mean(episode_rewards[-101:-1]), 1)
            print("********************************************************")
            print("steps: {}".format(t))
            print("episodes: {}".format(num_episodes))
            print("mean 100 episode reward: {}".format(mean_100ep_reward))
            print("% time spent exploring: {}".format(int(100 * eps_threshold)))
            print("********************************************************")





********************************************************
steps: 225
episodes: 10
mean 100 episode reward: 2.6
% time spent exploring: 99
********************************************************
********************************************************
steps: 595
episodes: 20
mean 100 episode reward: 3.2
% time spent exploring: 99
********************************************************
********************************************************
steps: 883
episodes: 30
mean 100 episode reward: 2.7
% time spent exploring: 99
********************************************************
********************************************************
steps: 1356
episodes: 40
mean 100 episode reward: 3.4
% time spent exploring: 98
********************************************************
********************************************************
steps: 1915
episodes: 50
mean 100 episode reward: 3.9
% time spent exploring: 98
********************************************************
****************************